In [1]:
!pip install plotly --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 51.5 MB/s eta 0:00:00
  Attempting uninstall: plotly
    Found existing installation: plotly 5.24.1
    Uninstalling plotly-5.24.1:
      Successfully uninstalled plotly-5.24.1


In [7]:
!pip install dash --quiet

In [8]:
import plotly.graph_objects as go
import plotly.express as px

In [9]:
!wget "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv"



--2025-07-06 20:05:01--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2476 (2.4K) [text/csv]
Saving to: ‘spacex_launch_dash.csv.1’

spacex_launch_dash. 100%[===================>]   2.42K  --.-KB/s    in 0s      

2025-07-06 20:05:01 (626 MB/s) - ‘spacex_launch_dash.csv.1’ saved [2476/2476]



In [10]:
!wget "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/t4-Vy4iOU19i8y6E3Px_ww/spacex-dash-app.py"



--2025-07-06 20:05:02--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/t4-Vy4iOU19i8y6E3Px_ww/spacex-dash-app.py
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2075 (2.0K) [application/x-python]
Saving to: ‘spacex-dash-app.py.1’

spacex-dash-app.py. 100%[===================>]   2.03K  --.-KB/s    in 0s      

2025-07-06 20:05:02 (786 MB/s) - ‘spacex-dash-app.py.1’ saved [2075/2075]



In [11]:
from dash import Dash, dcc, html, Input, Output
from dash.dash_table import DataTable
from dash import jupyter_dash

In [12]:
!pip install jupyter-dash --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 65.6 MB/s eta 0:00:00


In [13]:
from dash import dcc, html, Input, Output
import pandas as pd
from dash import Dash

# Load data
spacex_df = pd.read_csv("spacex_launch_dash.csv")
unique_sites = sorted(spacex_df['Launch Site'].unique())

dropdown_options = [{'label': 'All Sites', 'value': 'ALL'}] + \
                   [{'label': site, 'value': site} for site in unique_sites]

# ✅ Use Dash here
app = Dash(__name__)
app.layout = html.Div([
    html.H1("SpaceX Launch Records Dashboard", style={'textAlign': 'center', 'color': '#503D36',
                                                      'font-size': 40}),

    dcc.Dropdown(
        id='site-dropdown',
        options=dropdown_options,
        value='ALL',
        placeholder="Select a Launch Site here",
        searchable=True,
        style={'width': '80%', 'padding': '3px', 'font-size': '20px', 'textAlign': 'center', 'margin': 'auto'}
    ),

    html.Br(),
    html.Div(id='output-container')
])

# ✅ Inline display
app.run(mode='inline')

<IPython.core.display.Javascript object>

In [14]:
app.layout = html.Div([
    html.H1("SpaceX Launch Records Dashboard", style={'textAlign': 'center', 'color': '#503D36',
                                                      'font-size': 40}),

    dcc.Dropdown(
        id='site-dropdown',
        options=dropdown_options,
        value='ALL',
        placeholder="Select a Launch Site here",
        searchable=True,
        style={'width': '80%', 'padding': '3px', 'font-size': '20px', 'textAlign': 'center', 'margin': 'auto'}
    ),

    html.Br(),

    # ✅ Add the pie chart here
    dcc.Graph(id='success-pie-chart'),

    html.Br(),
    html.Div(id='output-container')
])

In [15]:
from dash.dependencies import Input, Output
import plotly.express as px

@app.callback(
    Output(component_id='success-pie-chart', component_property='figure'),
    Input(component_id='site-dropdown', component_property='value')
)
def update_pie_chart(selected_site):
    if selected_site == 'ALL':
        # Total success launches by site
        fig = px.pie(
            spacex_df,
            names='Launch Site',
            values='class',
            title='Total Success Launches by Site'
        )
    else:
        # Filter the dataframe for selected site
        filtered_df = spacex_df[spacex_df['Launch Site'] == selected_site]

        # Count success and failure
        success_counts = filtered_df['class'].value_counts().reset_index()
        success_counts.columns = ['class', 'count']
        success_counts['class'] = success_counts['class'].replace({1: 'Success', 0: 'Failure'})

        # Create pie chart
        fig = px.pie(
            success_counts,
            names='class',
            values='count',
            title=f'Success vs Failure for {selected_site}'
        )
    return fig

In [16]:
app.run(mode='inline')

<IPython.core.display.Javascript object>

In [17]:
min_payload = spacex_df['Payload Mass (kg)'].min()
max_payload = spacex_df['Payload Mass (kg)'].max()

In [18]:
html.P("Payload range (Kg):", style={'fontWeight': 'bold'}),

dcc.RangeSlider(
    id='payload-slider',
    min=0,
    max=10000,
    step=1000,
    value=[min_payload, max_payload],
    marks={0: '0', 2500: '2500', 5000: '5000', 7500: '7500', 10000: '10000'},
    tooltip={"placement": "bottom", "always_visible": True},
    # style={'padding': '40px'} # Removed style argument
),

(RangeSlider(min=0, max=10000, step=1000, marks={0: '0', 2500: '2500', 5000: '5000', 7500: '7500', 10000: '10000'}, value=[0.0, 9600.0], tooltip={'placement': 'bottom', 'always_visible': True}, id='payload-slider'),)

In [19]:
app.layout = html.Div([
    html.H1("SpaceX Launch Records Dashboard", style={'textAlign': 'center', 'color': '#503D36',
                                                      'font-size': 40}),

    dcc.Dropdown(
        id='site-dropdown',
        options=dropdown_options,
        value='ALL',
        placeholder="Select a Launch Site here",
        searchable=True,
        style={'width': '80%', 'padding': '3px', 'font-size': '20px', 'textAlign': 'center', 'margin': 'auto'}
    ),

    html.Br(),
    dcc.Graph(id='success-pie-chart'),

    html.Br(),
    html.P("Payload range (Kg):", style={'fontWeight': 'bold'}),

    dcc.RangeSlider(
        id='payload-slider',
        min=0,
        max=10000,
        step=1000,
        value=[min_payload, max_payload],
        marks={0: '0', 2500: '2500', 5000: '5000', 7500: '7500', 10000: '10000'},
        tooltip={"placement": "bottom", "always_visible": True},
        # style={'padding': '40px'} # Removed style argument
    ),

    html.Br(),
    html.Div(id='output-container')
])

In [20]:
html.Div([
    html.P("Payload range (Kg):", style={'fontWeight': 'bold'}),

    html.Div([
        dcc.RangeSlider(
            id='payload-slider',
            min=0,
            max=10000,
            step=1000,
            value=[min_payload, max_payload],
            marks={
                0: '0',
                2500: '2500',
                5000: '5000',
                7500: '7500',
                10000: '10000'
            },
            tooltip={"placement": "bottom", "always_visible": True}
        )
    ], style={'width': '100%', 'padding': '0px 30px 30px 30px'})
])

Div([P(children='Payload range (Kg):', style={'fontWeight': 'bold'}), Div(children=[RangeSlider(min=0, max=10000, step=1000, marks={0: '0', 2500: '2500', 5000: '5000', 7500: '7500', 10000: '10000'}, value=[0.0, 9600.0], tooltip={'placement': 'bottom', 'always_visible': True}, id='payload-slider')], style={'width': '100%', 'padding': '0px 30px 30px 30px'})])

In [21]:
app.run(mode='inline')

<IPython.core.display.Javascript object>

In [22]:
dcc.Graph(id='success-payload-scatter-chart'),

(Graph(id='success-payload-scatter-chart'),)

In [23]:
from dash.dependencies import Input, Output
import plotly.express as px

@app.callback(
    Output(component_id='success-payload-scatter-chart', component_property='figure'),
    Input(component_id='site-dropdown', component_property='value'),
    Input(component_id='payload-slider', component_property='value')
)
def update_scatter(selected_site, payload_range):
    low, high = payload_range

    # Filter by payload range first
    filtered_df = spacex_df[
        (spacex_df['Payload Mass (kg)'] >= low) &
        (spacex_df['Payload Mass (kg)'] <= high)
    ]

    if selected_site == 'ALL':
        fig = px.scatter(
            filtered_df,
            x='Payload Mass (kg)',
            y='class',
            color='Booster Version Category',
            title='Correlation Between Payload and Success for All Sites',
            hover_data=['Launch Site']
        )
    else:
        # Filter by selected launch site
        site_df = filtered_df[filtered_df['Launch Site'] == selected_site]
        fig = px.scatter(
            site_df,
            x='Payload Mass (kg)',
            y='class',
            color='Booster Version Category',
            title=f'Correlation Between Payload and Success for site {selected_site}',
            hover_data=['Launch Site']
        )

    return fig

In [24]:
app.layout = html.Div([
    html.H1("SpaceX Launch Records Dashboard", style={'textAlign': 'center'}),

    # ✅ Valid dropdown component
    dcc.Dropdown(
        id='site-dropdown',
        options=dropdown_options,
        value='ALL',
        placeholder="Select a Launch Site here",
        searchable=True,
        style={'width': '80%', 'padding': '3px', 'font-size': '20px', 'textAlign': 'center', 'margin': 'auto'}
    ),

    dcc.Graph(id='success-pie-chart'),

    html.Div([
        html.P("Payload range (Kg):", style={'fontWeight': 'bold'}),
        html.Div([
            dcc.RangeSlider(
                id='payload-slider',
                min=0,
                max=10000,
                step=1000,
                value=[min_payload, max_payload],
                marks={0: '0', 2500: '2500', 5000: '5000', 7500: '7500', 10000: '10000'},
                tooltip={"placement": "bottom", "always_visible": True}
            )
        ], style={'width': '100%', 'padding': '0px 30px 30px 30px'})
    ]),

    dcc.Graph(id='success-payload-scatter-chart')
])



In [25]:
app.run(mode='inline')

<IPython.core.display.Javascript object>

In [26]:
spacex_df[spacex_df['class'] == 1]['Launch Site'].value_counts()

,count
Launch Site,
KSC LC-39A,10
CCAFS LC-40,7
VAFB SLC-4E,4
CCAFS SLC-40,3


In [27]:
success_rate = spacex_df.groupby('Launch Site')['class'].mean().sort_values(ascending=False)
success_rate

,class
Launch Site,
KSC LC-39A,0.769231
CCAFS SLC-40,0.428571
VAFB SLC-4E,0.400000
CCAFS LC-40,0.269231


In [28]:
spacex_df['payload_bin'] = pd.cut(spacex_df['Payload Mass (kg)'],
                                   bins=[0, 2000, 4000, 6000, 8000, 10000],
                                   labels=['0–2k', '2–4k', '4–6k', '6–8k', '8–10k'])

spacex_df.groupby('payload_bin')['class'].mean().sort_values(ascending=False)

/tmp/ipython-input-28-3059316340.py:5: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



,class
payload_bin,
2–4k,0.619048
8–10k,0.600000
4–6k,0.384615
0–2k,0.272727
6–8k,0.000000


In [29]:
spacex_df.groupby('Booster Version Category')['class'].mean().sort_values(ascending=False)

,class
Booster Version Category,
B5,1.000000
FT,0.666667
B4,0.545455
v1.1,0.066667
v1.0,0.000000


In [30]:
print("1. Most successful launches by site:\n", spacex_df[spacex_df['class'] == 1]['Launch Site'].value_counts(), "\n")

print("2. Highest success rate by site:\n", spacex_df.groupby('Launch Site')['class'].mean().sort_values(ascending=False), "\n")

spacex_df['payload_bin'] = pd.cut(spacex_df['Payload Mass (kg)'], bins=[0, 2000, 4000, 6000, 8000, 10000],
                                   labels=['0–2k', '2–4k', '4–6k', '6–8k', '8–10k'])

print("3. Success rate by payload range:\n", spacex_df.groupby('payload_bin')['class'].mean().sort_values(ascending=False), "\n")

print("4. Success rate by booster version:\n", spacex_df.groupby('Booster Version Category')['class'].mean().sort_values(ascending=False), "\n")

1. Most successful launches by site:
 Launch Site
KSC LC-39A      10
CCAFS LC-40      7
VAFB SLC-4E      4
CCAFS SLC-40     3
Name: count, dtype: int64 

2. Highest success rate by site:
 Launch Site
KSC LC-39A      0.769231
CCAFS SLC-40    0.428571
VAFB SLC-4E     0.400000
CCAFS LC-40     0.269231
Name: class, dtype: float64 

3. Success rate by payload range:
 payload_bin
2–4k     0.619048
8–10k    0.600000
4–6k     0.384615
0–2k     0.272727
6–8k     0.000000
Name: class, dtype: float64 

4. Success rate by booster version:
 Booster Version Category
B5      1.000000
FT      0.666667
B4      0.545455
v1.1    0.066667
v1.0    0.000000
Name: class, dtype: float64 



/tmp/ipython-input-30-2910124314.py:8: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

